In [0]:
S_t-state
a_t=action
r_t+1=reward at next state
alpha-learning rate
gamma-discount
epsilon-exploration
Q-value-->maximum reward that can be taken in a state
Q(s_t,a_t)=(r_t+1)+gamma(r_t+2)+gamma^2(r_t+3)+gamma^3(r_t+4)+........
Q(s_t+1,a_t+1)=(r_t+2)+gamma(r_t+3)+gamma^2(r_t+4)+gamma^3(r_t+5)+........
Q(s_t,a_t)=r_t+1 + gammaQ(s_t+1,a_t+1)
Q(s_t,a_t)=r_t+1 + gamma*Q_max(s_t+1)

In [0]:
 ___________
|__|__|__|__|      16 columns determines state,4 rows determines(action ie. top,down,left,right)
|__|__|__|__|
|__|__|__|__|=====>
|__|__|__|__|

S-start
H-Hole
F-forzen
G-gole

In [0]:
from IPython.display import clear_output

In [0]:
import gym
import random
import numpy as np
import time

In [0]:
from gym.envs.registration import register

In [0]:
#https://github.com/openai/gym/blob/master/gym/envs/__init__.py

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4','is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env_name='FrozenLakeNoSlip-v0'
env=gym.make(env_name)

In [0]:
env.observation_space

Discrete(16)

In [0]:
env.action_space

Discrete(4)

In [0]:
type(env.action_space)

gym.spaces.discrete.Discrete

In [0]:
class  Agent():
    def __init__(self,env):
        self.env=env
        self.actionSize=env.action_space.n   #4
        self.stateSize=env.observation_space.n  #16
        
    def getAction(self,state):
        action=self.env.action_space.sample()
        #or  action=random.choice(range(self.actionSize))
        
        #action=np.random.uniform(self.env.action_space.low,
        #                       self.env.action_space.high,         for continuous
        #                      self.env.action_space.shape)
        
        return action
agent=Agent(env)

In [0]:
for ep in range(20):
    state=env.reset()
    done=False
    while not done:
        action=agent.getAction(state)
        state,reward,done,info=env.step(action)
        env.render()
        time.sleep(0.05)
        clear_output(wait=True)
        #if reward==1:
         #   break
env.close()

  (Right)
SFFF
FHFH
FFFH
HFFG


# Q-Learning

In [0]:
class  Agent():
    def __init__(self,env,discountRate=0.97,learningRate=0.01):
        self.env=env
        self.actionSize=env.action_space.n   #4
        self.stateSize=env.observation_space.n  #16
        
        self.explorationRate=1.0
        self.discountRate=discountRate
        self.learningRate=learningRate
        
        self.qTable=1e-4*np.random.random([self.stateSize, self.actionSize])
        
    def getAction(self,state):
        qState=self.qTable[state]
        actionGreedy=np.argmax(qState)
        actionRandom=self.env.action_space.sample()
        if random.random()<self.explorationRate:
            action=actionRandom 
        else:
            action=actionGreedy
        return action
    
    def train(self,state,action,nextState,reward,done):
        if done:
            qNext=np.zeros([self.actionSize])
        else:
            qNext=self.qTable[nextState]
        qTarget=reward+self.discountRate*np.max(qNext)
        qUpdate=qTarget-self.qTable[state,action]
        self.qTable[state,action]+=self.learningRate*qUpdate
        
        if done:
            self.explorationRate*=0.99
            
agent=Agent(env)

In [0]:
totalReward=0
for ep in range(100):
    state=env.reset()
    done=False
    while not done:
        action=agent.getAction(state)
        nextState,reward,done,info=env.step(action)
        agent.train(state, action, nextState, reward, done)
        
        print(f's:{state},a:{action}')
        state=nextState
        
        totalReward+=reward
        print(f'Ep:{ep},Goal:{totalReward},Explore:{agent.explorationRate}')
        
        env.render()
        print(agent.qTable)
        time.sleep(0.05)
        clear_output(wait=True)
env.close()

s:14,a:2
Ep:99,Goal:43.0,Explore:0.13397967485796175
  (Right)
SFFF
FHFH
FFFH
HFFG
[[7.56395168e-05 9.45823816e-05 7.41979865e-05 7.61343662e-05]
 [8.08994251e-05 2.49032650e-05 7.44601852e-05 6.57981636e-05]
 [3.85665267e-05 5.43058199e-05 3.63199902e-05 3.59834771e-05]
 [1.95018261e-05 6.46909715e-05 2.40466680e-05 6.90188723e-05]
 [6.52149870e-05 2.83977460e-04 1.10431550e-05 7.63533275e-05]
 [3.05358414e-05 4.44658280e-05 1.91291334e-05 5.50356009e-06]
 [7.31174695e-05 6.50897838e-04 5.12037007e-05 6.98077298e-05]
 [9.41768709e-05 4.91222028e-05 3.46179454e-05 2.33618581e-05]
 [7.76734358e-05 2.78592869e-05 1.87930899e-03 3.81886872e-05]
 [6.62668179e-05 1.30740151e-05 1.26477369e-02 7.00684977e-05]
 [1.93623472e-04 7.96018428e-02 5.95309917e-05 8.03073106e-05]
 [3.92084674e-05 7.29395092e-05 1.28698768e-05 7.63095707e-05]
 [7.77027535e-05 7.13308314e-06 9.39821301e-05 3.68425613e-05]
 [4.09219683e-05 4.94298200e-05 3.65103299e-05 2.06698445e-04]
 [5.18579145e-05 4.67627893e-03 3.7

#  Q-Learning with Neural Network

In [0]:
import tensorflow as tf

In [0]:
class  Agent():
    def __init__(self,env,discountRate=0.97,learningRate=0.01):
        self.env=env
        self.actionSize=env.action_space.n   #4
        self.stateSize=env.observation_space.n  #16
        
        self.explorationRate=1.0
        self.discountRate=discountRate
        self.learningRate=learningRate
        
        tf.reset_default_graph()
        self.stateIn=tf.placeholder(tf.int32,shape=[1])
        self.actionIn=tf.placeholder(tf.int32,shape=[1])
        self.targetIn=tf.placeholder(tf.float32,shape=[1])
        
        self.state=tf.one_hot(self.stateIn,depth=self.stateSize)
        self.action=tf.one_hot(self.actionIn,depth=self.actionSize)
        
        self.qState=tf.layers.dense(self.state,units=self.actionSize, name='QTable')
        self.qAction=tf.reduce_sum(tf.multiply(self.qState,self.action),axis=1)
        
        self.loss=tf.reduce_sum(tf.square(self.targetIn-self.qAction))
        self.optimizer=tf.train.AdamOptimizer(self.learningRate).minimize(self.loss)
        
        self.sess=tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
    def getAction(self,state):
        qState=self.sess.run(self.qState, feed_dict={self.stateIn:[state]})
        actionGreedy=np.argmax(qState)
        actionRandom=self.env.action_space.sample()
        if random.random()<self.explorationRate:
            action=actionRandom 
        else:
            action=actionGreedy
        return action
    
    def train(self,state,action,nextState,reward,done):
        if done:
            qNext=np.zeros([self.actionSize])
        else:
            qNext=self.sess.run(self.qState, feed_dict={self.stateIn:[nextState]})
        qTarget=reward+self.discountRate*np.max(qNext)
        
        feed={
            self.stateIn:[state],
            self.actionIn:[action],
            self.targetIn:[qTarget]
        }
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if done:
            self.explorationRate*=0.99
        
    def __del__(self):
        self.sess.close()
            
agent=Agent(env)

In [0]:
totalReward=0
for ep in range(100):
    state=env.reset()
    done=False
    while not done:
        action=agent.getAction(state)
        nextState,reward,done,info=env.step(action)
        agent.train(state, action, nextState, reward, done)
        
        print(f's:{state},a:{action}')
        state=nextState
        
        totalReward+=reward
        print(f'Ep:{ep},Goal:{totalReward},Explore:{agent.explorationRate}')
        
        env.render()
        with tf.variable_scope('QTable', reuse=True):
            weights=agent.sess.run(tf.get_variable('kernel'))
            print(weights)
        
        time.sleep(0.05)
        clear_output(wait=True)
        
env.close()

s:14,a:2
Ep:99,Goal:17.0,Explore:0.36603234127322926
  (Right)
SFFF
FHFH
FFFH
HFFG
[[ 0.2803828   0.35699728  0.18399435  0.35858876]
 [ 0.28144774 -0.5374817   0.15907648  0.27208474]
 [ 0.33027396 -0.10502584  0.16348262  0.36058888]
 [ 0.11249182 -0.2907312   0.21272534  0.1573511 ]
 [ 0.31486636  0.38480148 -0.647843    0.37505475]
 [-0.39050534 -0.31017357 -0.52743286 -0.28745803]
 [-0.39061326 -0.3525641  -0.2906056   0.3319691 ]
 [ 0.4671831   0.06217569  0.36608124 -0.10325885]
 [ 0.3358321  -0.48684606  0.26382816  0.41029578]
 [ 0.32928905  0.45160905  0.2800546  -0.41874123]
 [ 0.37605     0.48248896  0.18519376  0.35058293]
 [ 0.44320905  0.4936825   0.45347178 -0.16480187]
 [-0.14257866  0.0737763   0.27911562  0.41187555]
 [-0.5435657   0.4504788   0.33276573  0.46915168]
 [ 0.39861926  0.4466499   0.36276028  0.44972998]
 [ 0.21421516  0.16343725  0.08921695 -0.34565684]]
